In [1]:
import pandas as pd   
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle


In [3]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# we wil be considering Estimated salary as the dependent features as it has continous values.

data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [5]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
onehot_encoder_geo = OneHotEncoder(handle_unknown = 'ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [7]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [12]:
# Split the dataset in training and testing sets
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

# Scale these features
scaler= StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)
    

In [15]:
# Train ANN with regression problem statement

import tensorflow as tf    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [20]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # output layer for reqression linear activation function is used as default
])

#compile the model

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

#setup Tensorboard
log_dir = 'regressionlogs/fit/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq = 1)

In [22]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [23]:
history = model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 1s 2ms/step - loss: 100381.3516 - mae: 100381.3516 - val_loss: 98528.8438 - val_mae: 98528.8438
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 99651.2188 - mae: 99651.2188 - val_loss: 97036.1719 - val_mae: 97036.1719
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 96997.7422 - mae: 96997.7422 - val_loss: 93118.5312 - val_mae: 93118.5312
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 91691.3672 - mae: 91691.3672 - val_loss: 86505.3203 - val_mae: 86505.3203
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 83890.0859 - mae: 83890.0859 - val_loss: 77925.4062 - val_mae: 77925.4062
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 74754.3594 - mae: 74754.3594 - val_loss: 69090.8516 - val_mae: 69090.8516
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 66045.5000 - mae: 66045.500

In [24]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 22580), started 0:00:08 ago. (Use '!kill 22580' to kill it.)

In [ ]:
#Evaluate model om testdata

test_loss, test_mae =model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

train_loss = history.history['loss'][-1]
print(f'Train loss : {train_loss}')

val_loss = history.history['val_loss'][-1]
print("Validation Loss:", val_loss)

63/63 [==============================] - 0s 671us/step - loss: 50292.0703 - mae: 50292.0703
Test MAE : 50292.0703125
Train loss : 49412.11328125
Validation Loss: 50316.0703125


: 

In [29]:
model.save('regression_model.h5')

d:\Projects\churn_rate_ann\m\myenvname\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
